In [1]:
!pip install pandas ollama tqdm


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
PROMPT_TEMPLATE = """Instruction: Analyze the following review text and provide one distinct outputs formatted in JSON:

1. **Sentiment Classification:** Indicate whether the sentiment of the review is "positive" or "negative".
2. **Named Entity Extraction:** List all named entities present in the text, categorizing them by label (PERSON, ORG, LOC).
3. **Required JSON Format:** Ensure the response is formatted in JSON according to the following schema:

{{
  "sentiment": "<sentiment>",
  "review": "<review>",
  "entities": [
    {{
      "label": "<label>",
      "value": "<value>"
    }}
  ]
}}

example:

"I recently visited the restaurant 'La Dolce Vita' in Rome and was thrilled with the service and food. The waiter, Marco, was exceptionally friendly and the truffle risotto was simply divine. I can't wait to return and recommend this place to my friends."

```json
{{
  "sentiment": "positive",
  "review": "I recently visited the restaurant 'La Dolce Vita' in Rome and was thrilled with the service and food. The waiter, Marco, was exceptionally friendly and the truffle risotto was simply divine. I can't wait to return and recommend this place to my friends.",
  "entities": [
    {{
      "label": "ORG",
      "value": "La Dolce Vita"
    }},
    {{
      "label": "LOC",
      "value": "Rome"
    }},
    {{
      "label": "PERSON",
      "value": "Marco"
    }}
  ]
}}
```

{content}"""

In [6]:
import ollama

def process_review(review: str, model_name: str) -> str:
    """
    It processes the review text and returns the LLM response as string.
    
    Arguments:
        review (str): The review text.
        
    Return:
        The LLM response as string.
    """
    try:
        return ollama.generate(
            model=model_name,
            prompt=PROMPT_TEMPLATE.format(content=review)
        )["response"]
    except Exception as e:
        logging.error(f"Error invoking the chain: {e}")
        print(e)
        return None

In [12]:
import pandas as pd
from tqdm import tqdm

def call_model_llm(model_name: str, output_file_path: str) -> pd.DataFrame:
    """
    It calls the LLM model using Ollama. It returns the sampled dataframe enriched with relevant columns.
    
    Arguments:
        model_name: The name of the model to invoke via Ollama.
    
    Return:
        The enriched dataframe.
    """
    dataframe: pd.DataFrame = pd.read_csv(output_file_path)
    already_done_part: pd.DataFrame = dataframe[dataframe.output != "$$$"]
    slice_to_work_on: pd.DataFrame = dataframe[dataframe.output == "$$$"]
    total_rows: int = len(slice_to_work_on)
    for i in tqdm(range(total_rows), total=total_rows):
        row = slice_to_work_on.iloc[i]
        # logging.info(f"Processing row {i + 1} out of {total_rows}")
        slice_to_work_on.loc[i, "output"] = process_review(row["review"], model_name)
        pd.concat([already_done_part, slice_to_work_on])\
            .to_csv(output_file_path, index=False)
    return dataframe

In [ ]:
import os

# Define the model name to use
MODEL_NAME: str = "phi3:medium"

# Define the output path and get the result as csv
output_file_path: str = f"../resources/sampled_reviews_with_output_{MODEL_NAME.replace(':', '_')}.csv"

exists: bool = os.path.exists(output_file_path)
if not exists:
    sampled: pd.DataFrame = pd.read_csv("../resources/IMDB Dataset Sampled.csv")
    sampled["output"] = sampled.apply(lambda row: "$$$", axis=1)
    sampled.to_csv(output_file_path, index=False)

# Produce a dataframe by invoking the chain
output: pd.DataFrame = call_model_llm(model_name=MODEL_NAME, output_file_path=output_file_path)

output.to_csv(output_file_path, index=False)

logging.info("Completed processing.")

  0%|          | 0/999 [00:00<?, ?it/s]